In [1]:
import pandas as pd
import re

In [3]:
data = pd.read_csv(r"2022_01_22_MF_all_year_data_without_duplicates_drop142_new.csv", encoding = 'latin1', low_memory = False)

In [4]:
data.shape

(112599, 699)

In [5]:
data.columns

Index(['Unnamed: 0', 'accession#', 'filing_date', 'fund_name', 'fund_CIK',
       'stock_series#', 'principal_risks', 'principal_strategies',
       'business_address', 'mail_address',
       ...
       'Unnamed: 689', 'Unnamed: 690', 'Unnamed: 691', 'Unnamed: 692',
       'Unnamed: 693', 'Unnamed: 694', 'Unnamed: 695', 'Unnamed: 696',
       'Unnamed: 697', 'Unnamed: 698'],
      dtype='object', length=699)

In [14]:
data1 = data.iloc[:,0:13]
data1 = data1[pd.to_numeric(data1['Unnamed: 0'], errors='coerce').notnull()]
data1 = data1[pd.to_numeric(data1['filing_date'], errors='coerce').notnull()]
data1 = data1[pd.to_numeric(data1['fund_CIK'], errors='coerce').notnull()]
data1 = data1[pd.to_numeric(data1['filing_year'], errors='coerce').notnull()]
data1.columns

Index(['Unnamed: 0', 'accession#', 'filing_date', 'fund_name', 'fund_CIK',
       'stock_series#', 'principal_risks', 'principal_strategies',
       'business_address', 'mail_address', 'crsp_obj', 'crsp_class',
       'filing_year'],
      dtype='object')

In [15]:
data1.shape

(105514, 13)

In [16]:
data1['filing_year'].value_counts()

2020    10164
2019    10031
2018     9938
2017     9793
2016     9657
2015     9345
2014     8917
2021     8774
2013     8467
2012     7809
2011     7168
2010     5451
Name: filing_year, dtype: int64

In [17]:
data1.to_csv('2022_01_22_MF_all_year_data_without_duplicates_drop142_cleaned.csv', index = False)

In [19]:
data1 = data1.dropna(subset=['principal_risks'])

### Extract Sentences

In [20]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [21]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [22]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    contractions = abbreviations.keys()
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    for sentence_terminator in sentence_terminators:
        t_indices = list(find_all(paragraph, sentence_terminator))
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    for contraction in contractions:
        c_indices = list(find_all(paragraph, contraction))
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    return end

In [23]:
def find_sentences(paragraph):
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    for items in replacingList:
        paragraph = paragraph.replace(items[0], items[1])
    paragraph = re.sub(' +', ' ', paragraph)
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]
    end = True
    sentences = []
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    sentences.append(paragraph)
    sentences.reverse()
    return sentences

### input paragraphs, then split into sentences by using the above function find_sentences()

In [24]:
data1['cleaned_principal_risks']= ""
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    para = data1['principal_risks'].iloc[row]
    sents = find_sentences(para)
    data1['cleaned_principal_risks'].iloc[row] = sents

processing row:  0


C:\Users\Akshay\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000


In [25]:
data1.shape

(111367, 700)

In [26]:
data1.head(1)

,Unnamed: 0,accession#,filing_date,fund_name,fund_CIK,stock_series#,principal_risks,principal_strategies,business_address,mail_address,...,Unnamed: 690,Unnamed: 691,Unnamed: 692,Unnamed: 693,Unnamed: 694,Unnamed: 695,Unnamed: 696,Unnamed: 697,Unnamed: 698,cleaned_principal_risks
0,57167,0001104659-10-000128,20100104,ssga core edge equity fund,826686,s000018548,it is possible to lose money by investing in t...,ssga core edge equity fund invests at least 80...,STREET 1: 909 A ST CITY: TACOMA STATE: WA ZIP...,STREET 1: 909 A STREET CITY: TACOMA STATE: WA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[it is possible to lose money by investing in ...


In [27]:
type(data1['cleaned_principal_risks'].iloc[0])

list

In [29]:
data1.to_csv('2022_01_22_principal_risks_cleaned_non_dedup_1.csv')

### Public Health Keywords to Search Sentences that Contain one or multiple of these keywords

In [30]:
# Public Health keywords: 
ph_word = ['communicable diseases','health crises','pandemic','respiratory','illness','prevention','epidemic',
           'coronavirus','health crisis','pandemics','sanitation','global health crises',
           'covid','health screenings','pathogens','sars','global health crisis',
           'covid 19','hiv','preparedness','sars cov 2','epidemics','disease','influenza',
           'public health','virus','global health','diseases','mers','quarantines','h1n1','viruses']

ph_word = [ 'communicable   diseases',         
            'coronavirus',                          
            'covid','covid 19',                     
            'disease',                              
            'epidemic',                             
            'global health',                        
            'global health crisis',                 
            'health crises',                        
            'health crisis' ,                       
            'health screenings' ,                   
            'hiv' ,                                 
            'influenza', 'h1n1',                     
            'pandemic','epidemic',             
            'pathogens',                            
            'prevention',                           
            'public health',                        
            'quarantines',                          
            'respiratory illness',                  
            'sanitation',                           
            'sars', 'sars cov 2', 'mers',        
            'virus']

ph_word = [ 'communicable diseases',
            'coronavirus',
            'covid','covid 19',
            'disease',
            'epidemic',
            'global health',
            'global health crisis',
            'health crises',
            'health crisis' ,
            'health screenings' ,
            'hiv' ,
            'influenza', 'h1n1',
            'pandemic','epidemic',
            'pathogens',
            'prevention',
            'public health',
            'quarantines',
            'respiratory illness',
            'sanitation',
            'sars', 'sars cov 2', 'mers',
            'virus','COVID','COVID-','COVID –','Virus','Outbreak','Global pandemic','Vaccine','Delta','Omicron','Variant','Lockdown','closure /s COVID',
            'case count /s COVID','case /s COVID','epidemic','endemic',
            'death toll /s COVID',
            'infectious disease risk',
            'coronavirus',
            'economic disruption /s COVID',
            'vaccine hesitancy',
            'vaccine skepticism',
            'vaccine opposition',
            'vaccine rollout',
            'vaccine mandate',
            'vaccine passport',
            'novel virus',
            'COVID measures' ]


In [31]:
import os
import pandas as pd
import glob
import nltk.data
import re
#from nltk.tokenize import tokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktTrainer

pubic_health = pd.DataFrame()
ph_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in ph_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                ph_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
pubic_health = pd.DataFrame(ph_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])               

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000


In [32]:
pubic_health.shape

(10958, 4)

In [33]:
pubic_health.head()

,accession#,filing_year,fund_name,sentences
0,0000950123-10-006915,2010,bb&t large cap fund,etf risk: the value of the fundâs investment...
1,0001193125-10-044893,2010,aston/new century absolute return etf fund,the value of commodities and commodity contrac...
2,0001193125-10-045911,2010,aston dynamic allocation fund,the value of commodities and commodity contrac...
3,0000728889-10-000574,2010,oppenheimer real asset fund,prices of commodities and related contracts ma...
4,0001193125-10-071718,2010,aston/lake partners lasso alternatives fund,the value of commodities and commodity contrac...


In [36]:
pubic_health['filing_year'].value_counts()

2020    8912
2017     262
2016     242
2015     231
2018     230
2019     205
2014     173
2012     126
2013     126
2011     110
2010      56
2021      43
Name: filing_year, dtype: int64

In [38]:
pubic_health.to_excel("2021_07_public_health_keywords_sentences_dedup_23_01.xlsx")

In [39]:
 nd_word = [ 'cyclones',                   
             'death',                      
             'drought',                    
             'earthquakes',                
             'environmental damage',       
             'cloud',                      
             'disaster',                   
             'droughts',                   
             'earthquake',                 
             'fire',                       
             'flood',                      
             'hurricane',                  
             'hurricanes',                 
             'lightning',                  
             'natural disaster',           
             'natural disasters',          
             'seismic',                    
             'storms',                     
             'tornadoes',                  
             'tsunami',                    
             'underground',                
             'volcanoes',                  
             'windstorms']


In [40]:
nd_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in nd_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                nd_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
natural_disaster = pd.DataFrame(nd_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])   

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000
processing row:  110000


In [41]:
natural_disaster.shape

(8866, 4)

In [42]:
natural_disaster['filing_year'].value_counts()

2020    2067
2019     974
2018     962
2017     876
2016     806
2015     762
2014     577
2013     510
2012     449
2011     301
2010     144
2021      13
Name: filing_year, dtype: int64

In [43]:
natural_disaster.to_excel("2021_07_natural_disaster_keywords_sentences_dedup_23_01.xlsx")

In [44]:
pubic_health_non_dedup = pd.read_excel('2021_07_public_health_keywords_sentences_23_01.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '2021_07_public_health_keywords_sentences_23_01.xlsx'

In [ ]:
pubic_health_non_dedup.head(2)

In [ ]:
pubic_health_non_dedup['filing_year'].value_counts()